## MLMC Estimator Pretest - Proof of Concept

This notebook investigates the behaviour of different multilevel Monta Carlo extensions of the Harrell-Davis quantile estimator, specifically regarding their accuracy and variance.

To understand the characteristics of the proposed estimators, the QOI is chosen as a quantile of a well-known distribution - the normal distribution $N(\mu, \sigma^2)$.

### 1. Set the parameters

In [119]:
# Distribution of interest: 
mu = -10
sd = 200

# Quantile of interest:
p = 0.005

# Model valuation costs:
c_f = 1
c_c = c_f / 200

# Number of samples:
n_f = 50
n_c = 50000
n_std_mc = int((n_f * (c_f + c_c) + n_c * c_c) / c_f)

# Number of bootstrap samples:
n_bootstrap = 1000

### 2. Define the sampling functions

In [120]:
import numpy as np

def SampleLevel1(mu, sd, n_f):
    fineModelSamples = np.random.normal(mu, sd, n_f)
    eps = 50 * np.random.normal(0, 0.6, n_f)
    coarseModelSamples_lvl1 = fineModelSamples + eps
    return fineModelSamples, coarseModelSamples_lvl1, eps

def SampleLevel2(mu, sd, n_c):
    eps = 50 * np.random.normal(0, 0.6, n_c)
    coarseModelSamples_50000 = np.random.normal(mu, sd, n_c) + eps
    return coarseModelSamples_50000, eps

### 3. Define Harrell-Davis weighting function

In [121]:
import scipy as scp

def HarrellDavisWeighting(p, xSorted):
    n = xSorted.size
    hd = np.empty((2), np.float64)
    if n < 2:
        hd.flat = np.nan
        return hd[0]
    v = np.arange(n+1) / float(n)
    betacdf = scp.stats.distributions.beta.cdf
    _w = betacdf(v, (n+1)*p, (n+1)*(1-p))
    w = _w[1:] - _w[:-1]
    hd_mean = np.dot(w, xSorted)
    hd[0] = hd_mean
    hd[1] = np.dot(w, (xSorted-hd_mean)**2)
    return hd[0]

### 4. Define Quantile Estimators

In [122]:
from scipy.stats import norm, mstats
import pandas as pd

def CalcQuantileEstimates(p, p_f, p_c, mu, sd, fineSamples, coarseSamples_lvl1, coarseSamples_lvl2, eps2, numStdMCSamples):
    # Compute HD estimators
    hdEstimatorFineModel = mstats.hdquantiles(data=fineSamples, prob=(p_f), var=False).item()
    hdEstimatorCoarseModel_lvl1 = mstats.hdquantiles(data=coarseSamples_lvl1, prob=(p_f), var=False).item()
    hdEstimatorCoarseModel_lvl2 = mstats.hdquantiles(data=coarseSamples_lvl2, prob=(p_c), var=False).item()

    # Compute order statistics
    fineModelOrderStats = np.sort(fineSamples)
    coarseModelOrderStats_lvl1 = np.sort(coarseSamples_lvl1)
    coarseModelOrderStats_lvl2 = np.sort(coarseSamples_lvl2)

    # MLMC estimators
    #mlmcEstimator = hdEstimatorFineModel - hdEstimatorCoarseModel_lvl1 + hdEstimatorCoarseModel_lvl2
    mlmcEstimator = (HarrellDavisWeighting(p_f,fineModelOrderStats)
                     - HarrellDavisWeighting(p_f,coarseModelOrderStats_lvl1)
                     + HarrellDavisWeighting(p_c, coarseModelOrderStats_lvl2)
    )

    diffsOfOrderStats = np.sort(fineSamples) - np.sort(coarseSamples_lvl1)
    #mlmcEstimatorComb = (mstats.hdquantiles(data=diffs, prob=(p_f), var=False).item()
    #                     + mstats.hdquantiles(data=coarseSamples_lvl2, prob=(p_c), var=False).item()
    #)
    mlmcEstimatorComb = (HarrellDavisWeighting(p_f, diffsOfOrderStats)
                         + mstats.hdquantiles(data=coarseSamples_lvl2,
                                              prob=(p_c),
                                              var=False).item()
    )

    diffs = fineSamples - coarseSamples_lvl1
    orderStatsOfDiffs = np.sort(diffs)
    mlmcEstimatorApprox = (HarrellDavisWeighting(p_f, orderStatsOfDiffs)
                           + mstats.hdquantiles(data=coarseSamples_lvl2, prob=(p_c), var=False).item()
    )

    # Standard MC estimator
    fineModelSamples_stdMC = coarseSamples_lvl2[0:numStdMCSamples-1] - eps2[0:numStdMCSamples-1]
    hdEstimatorStandardMC = mstats.hdquantiles(data=fineModelSamples_stdMC,
                                               prob=(p),
                                               var=False).item()

    # Actual quantile
    actualQuantile = norm.ppf(p, loc=mu, scale=sd)

    # Combine into a DataFrame
    df = pd.DataFrame({
        "Value": [
            hdEstimatorFineModel,
            hdEstimatorCoarseModel_lvl1,
            hdEstimatorCoarseModel_lvl2,
            mlmcEstimator,
            mlmcEstimatorComb,
            mlmcEstimatorApprox,
            hdEstimatorStandardMC,
            actualQuantile
        ]
    })

    return df

### 5. Apply bootstrapping

In [123]:
estimators = [
    f"Fine Model HD, {n_f} samples",
    f"Coarse Model HD, {n_f} samples",
    f"Coarse Model HD, {n_c} samples",
    "MLMC (separate terms)",
    "MLMC (diff. of order stats)",
    "MLMC (order stats of diff.)",
    f"Standard MC-HD, {n_std_mc} samples",
    "Actual Quantile"
]

df = pd.DataFrame({"Estimator": estimators})

cols = []

for i in range(1, n_bootstrap + 1):

    fineModelSamples, coarseModelSamples_lvl1, eps1 = SampleLevel1(mu, sd, n_f)
    coarseModelSamples_lvl2, eps2 = SampleLevel2(mu, sd, n_c)

    col = CalcQuantileEstimates(
        p=p, p_f=p, p_c=p, mu=mu, sd=sd,
        fineSamples=fineModelSamples,
        coarseSamples_lvl1=coarseModelSamples_lvl1,
        coarseSamples_lvl2=coarseModelSamples_lvl2,
        eps2=eps2,
        numStdMCSamples=n_std_mc
    )["Value"]

    col.name = f"Run_{i}"
    cols.append(col)

new_cols = pd.concat(cols, axis=1)
df = pd.concat([df, new_cols], axis=1)

### 6. Compute bootstrapped statistics for the estimators

In [124]:
# Compute average across all Run columns
run_cols = [f"Run_{i}" for i in range(1, n_bootstrap+1)]
df["Mean"] = df[run_cols].mean(axis=1)

# Extract the Actual Quantile value from the DataFrame
actual_value = df.loc[df["Estimator"] == "Actual Quantile", "Mean"].values[0]

# Compute the deviation of the mean from the actual quantile
df["AbsDev"] = abs(df["Mean"] - actual_value)

# Compute standard deviation across all Run columns
df["StdDev"] = df[run_cols].std(axis=1)

# Identify all Run columns
run_cols = [col for col in df.columns if col.startswith("Run_")]

# Compute RMSE (RMS deviation from Actual Quantile)
df["RMSE"] = ((df[run_cols] - actual_value) ** 2).mean(axis=1) ** 0.5

# Show relevant columns
pd.set_option("display.precision", 3)
print(df[["Estimator", "Mean", "AbsDev", "StdDev", "RMSE"]])

                        Estimator     Mean  AbsDev     StdDev       RMSE
0       Fine Model HD, 50 samples -448.201  76.965  9.164e+01  1.196e+02
1     Coarse Model HD, 50 samples -452.978  72.188  9.193e+01  1.168e+02
2  Coarse Model HD, 50000 samples -531.193   6.027  4.314e+00  7.411e+00
3           MLMC (separate terms) -526.416   1.251  2.636e+01  2.638e+01
4     MLMC (diff. of order stats) -526.416   1.251  2.636e+01  2.638e+01
5     MLMC (order stats of diff.) -597.578  72.412  1.411e+01  7.377e+01
6     Standard MC-HD, 300 samples -536.891  11.726  5.409e+01  5.532e+01
7                 Actual Quantile -525.166   0.000  2.275e-13  8.640e-12


### Notations

- "MLMC (separate terms)" refers to the estimator

    $\sum_{i=1}^{n_{\text{f}}}{W_{i}^{n_{\text{f}}}X_{(i)}^{\text{f}}} - \sum_{i=1}^{n_{\text{f}}}{W_{i}^{n_{\text{f}}}X_{(i)}^{\text{c}}} + \sum_{i=1}^{n_{\text{c}}}{W_{i}^{n_{\text{c}}}X_{(i)}^{\text{c}}}$

- "MLMC (diff. of order stats)" refers to the estimator

    $\sum_{i=1}^{n_{\text{f}}}{W_{i}^{n_{\text{f}}}(X_{(i)}^{\text{f}} - X_{(i)}^{\text{c}})} + \sum_{i=1}^{n_{\text{c}}}{W_{i}^{n_{\text{c}}}X_{(i)}^{\text{c}}}$

- "MLMC (order stats of diff.)" refers to the estimator

    $\sum_{i=1}^{n_{\text{f}}}{W_{i}^{n_{\text{f}}}(X^{\text{f}} - X^{\text{c}})_{(i)}} + \sum_{i=1}^{n_{\text{c}}}{W_{i}^{n_{\text{c}}}X_{(i)}^{\text{c}}}$

### Observations

- Since the Harrell-Davis weights do not depend on the distribution but only on $p$ and $n$, "MLMC (separate terms)" and "MLMC (diff. of order stats)" yield the same result.

- If the difference between the costs $c_{\text{f}}$ and $c_{\text{c}}$ is large, "MLMC (diff. of order stats)" can yield significantly better results compared to standard HD Monte Carlo.

- For $c_{\text{c}} = \frac{c_{\text{f}}}{20}$, "MLMC (diff. of order stats)" and standard HD Monte Carlo yield similar results.

- If monotonicity between the fina and coarse model is introduced (i.e. the $\epsilon$ added to the fine model is always non-negative), the results of all MLMC estimators improve.

### Idea

The general approach is: 

\begin{align*} F^{-1}_{X^{\text{f}}}(p) &\approx \mathbb{E}[X^{\text{f}}_{((n+1)p)}] \\ &= \mathbb{E}[X^{\text{f}}_{(k)}] \\ &= \mathbb{E}[X^{\text{f}}_{(k)}] - \mathbb{E}[X^{\text{c}}_{(k)}] + \mathbb{E}[X^{\text{c}}_{(k)}] \\ &= \mathbb{E}[X^{\text{f}}_{((n_{\text{f}}+1)p_{\text{f}})}] - \mathbb{E}[X^{\text{c}}_{((n_{\text{f}}+1)p_{\text{f}})}] + \mathbb{E}[X^{\text{c}}_{((n_{\text{c}}+1)p_{\text{c}})}] \\ &= \dots \end{align*}

This uses:
1. $\mathbb{E}[X^{\text{f}}_{((n+1)p)}] \rightarrow F^{-1}_{X^{\text{f}}}(p)$ for $n \rightarrow \infty$, i.e. $n$ should be chosen large enough to approximate the quantile.

2. Define $k \coloneqq (n+1)p$.

3. Add zero.

4. Choose $n_{\text{f}}$ and $n_{\text{c}}$. Then set $p_{\text{f}} \coloneqq \frac{k}{n_{\text{f}}+1}$ and $p_{\text{c}} \coloneqq \frac{k}{n_{\text{c}}+1}$.

5. Derive HD-like estimator by:
    - "MLMC (separate terms)": applying Harrell-Davis to each summand separately

    - "MLMC (diff. of order stats)": applying Harrell-Davis to each summand separately, then using that the first two terms share the same HD weights.

    - "MLMC (order stats of diff.)": defining $Z \coloneqq X^{\text{f}} - X^{\text{c}}$ and using that 
    
        \begin{align*} \mathbb{E}[X^{\text{f}}_{((n_{\text{f}}+1)p_{\text{f}})}] - \mathbb{E}[X^{\text{c}}_{((n_{\text{f}}+1)p_{\text{f}})}] + \mathbb{E}[X^{\text{c}}_{((n_{\text{c}}+1)p_{\text{f}})}] &= \mathbb{E}[X^{\text{f}}_{((n_{\text{f}}+1)p_{\text{f}})} - X^{\text{c}}_{((n_{\text{f}}+1)p_{\text{f}})}] + \mathbb{E}[X^{\text{c}}_{((n_{\text{c}}+1)p_{\text{c}})}] \\ &= \mathbb{E}[(X^{\text{f}} - X^{\text{c}})_{((n_{\text{f}}+1)p_{\text{f}})}] + \mathbb{E}[X^{\text{c}}_{((n_{\text{c}}+1)p_{\text{c}})}] + \text{error} \\ &= \mathbb{E}[Z_{((n_{\text{f}}+1)p_{\text{f}})}] + \mathbb{E}[X^{\text{c}}_{((n_{\text{c}}+1)p_{\text{c}})}] + \text{error} \end{align*},
        
        then applying Harrell-Davis to the two summands separately.

### New Approach: Estimate different quantiles on the two levels ($p_{\text{f}}$, $p_{\text{c}}$ instead of $p$)

### 1. Set the parameters

In [125]:
# Distribution of interest: 
mu = -10
sd = 200

# Quantile of interest:
p = 0.005

# Model valuation costs:
c_f = 1
c_c = c_f / 200

# Number of samples:
n_std_mc = 260
n_f = 50
n_c = int(((n_std_mc - n_f) * c_f - n_f * c_c) / c_c)

# Quantiles to be estimated on the 2 levels
k = (n_std_mc + 1) * p
p_f = k / (n_f + 1)
p_c = k / (n_c + 1)
# Make sure p_f and p_c are in [0,1]

# Number of bootstrap samples:
n_bootstrap = 1000

### 2. Apply bootstrapping

In [126]:
estimators = [
    f"Fine Model HD, {n_f} samples",
    f"Coarse Model HD, {n_f} samples",
    f"Coarse Model HD, {n_c} samples",
    "MLMC (separate terms)",
    "MLMC (diff. of order stats)",
    "MLMC (order stats of diff.)",
    f"Standard MC-HD, {n_std_mc} samples",
    "Actual Quantile"
]

df = pd.DataFrame({"Estimator": estimators})

cols = []

for i in range(1, n_bootstrap + 1):

    fineModelSamples, coarseModelSamples_lvl1, eps1 = SampleLevel1(mu, sd, n_f)
    coarseModelSamples_lvl2, eps2 = SampleLevel2(mu, sd, n_c)

    col = CalcQuantileEstimates(
        p=p, p_f=p_f, p_c=p_c, mu=mu, sd=sd,
        fineSamples=fineModelSamples,
        coarseSamples_lvl1=coarseModelSamples_lvl1,
        coarseSamples_lvl2=coarseModelSamples_lvl2,
        eps2=eps2,
        numStdMCSamples=n_std_mc
    )["Value"]

    col.name = f"Run_{i}"
    cols.append(col)

new_cols = pd.concat(cols, axis=1)
df = pd.concat([df, new_cols], axis=1)

### 3. Compute bootstrapped statistics for the estimators

In [127]:
# Compute average across all Run columns
run_cols = [f"Run_{i}" for i in range(1, n_bootstrap+1)]
df["Mean"] = df[run_cols].mean(axis=1)

# Extract the Actual Quantile value from the DataFrame
actual_value = df.loc[df["Estimator"] == "Actual Quantile", "Mean"].values[0]

# Compute the deviation of the mean from the actual quantile
df["AbsDev"] = abs(df["Mean"] - actual_value)

# Compute standard deviation across all Run columns
df["StdDev"] = df[run_cols].std(axis=1)

# Identify all Run columns
run_cols = [col for col in df.columns if col.startswith("Run_")]

# Compute RMSE (RMS deviation from Actual Quantile)
df["RMSE"] = ((df[run_cols] - actual_value) ** 2).mean(axis=1) ** 0.5

# Show relevant columns
pd.set_option("display.precision", 3)
print(df[["Estimator", "Mean", "AbsDev", "StdDev", "RMSE"]])

                        Estimator     Mean   AbsDev     StdDev       RMSE
0       Fine Model HD, 50 samples -405.712  119.453  6.844e+01  1.377e+02
1     Coarse Model HD, 50 samples -410.247  114.918  6.984e+01  1.345e+02
2  Coarse Model HD, 41950 samples -827.923  302.757  4.233e+01  3.057e+02
3           MLMC (separate terms) -823.388  298.222  4.570e+01  3.017e+02
4     MLMC (diff. of order stats) -823.388  298.222  4.570e+01  3.017e+02
5     MLMC (order stats of diff.) -887.951  362.785  4.375e+01  3.654e+02
6     Standard MC-HD, 260 samples -535.408   10.242  5.842e+01  5.928e+01
7                 Actual Quantile -525.166    0.000  2.275e-13  8.640e-12
